Python Notebook by Kristoffer T. Bæk, 2022. 

In [ ]:
import os
import re
import pickle
import pandas as pd
from Bio import PDB, SeqIO
from Bio.PDB import PDBList, PDBParser, parse_pdb_header
from Bio import pairwise2
import freesasa as fs
from collections import namedtuple

### Functions

In [ ]:
def get_residueAreas(result):
    '''Extracts output values from freesasa and calculates RSA.'''            

    aa_codes = {'ALA': 'A',
             'CYS': 'C',
             'ASP': 'D',
             'GLU': 'E',
             'PHE': 'F',
             'GLY': 'G',
             'HIS': 'H',
             'ILE': 'I',
             'LYS': 'K',
             'LEU': 'L',
             'MET': 'M',
             'ASN': 'N',
             'PRO': 'P',
             'GLN': 'Q',
             'ARG': 'R',
             'SER': 'S',
             'THR': 'T',
             'VAL': 'V',
             'TRP': 'W',
             'TYR': 'Y'}

    l = []
    r = result.residueAreas()
    Sasa = namedtuple('Sasa', ['chain', 'number', 'residue', 'total_abs', 'total_rel', 'main_abs', 'main_rel'])
    
    for chain, chainvalue in r.items():
        for residue, value in chainvalue.items():
            if len(value.residueType) == 3: # disregard nucleotides
                residue_code = aa_codes[value.residueType]
                l.append(Sasa(chain,
                          str(value.residueNumber),
                          residue_code,
                          value.total,
                          value.relativeTotal,
                          value.mainChain,
                          value.relativeMainChain
                         ))

    return l

def calc_RSA(filepath):
    '''Runs freesasa on PDB.'''
    
    classifier = fs.Classifier.getStandardClassifier('naccess')
    structure = fs.Structure(filepath, classifier)
    result = fs.calc(structure)
    
    return get_residueAreas(result)

In [ ]:
def get_gaps(sequence):
     '''Identify gaps from alignments.'''

    for i, res in enumerate(sequence):
        if res not in ['-', 'X'] or i == len(sequence) - 1:
            gap_start = i
            break
    for i, res in enumerate(sequence[::-1]):
        if res not in ['-', 'X'] or i == len(sequence) - 1:
            gap_end = i
            break
    gap_mid = []
    gap = False
    end = len(sequence) - gap_end
    for i, res in enumerate(sequence[gap_start : end]):
        if res in ['-', 'X'] and not gap:
            start = i
            gap = True
        if res not in ['-', 'X'] and gap:
            end = i
            gap = False
            gap_mid.append(end - start)
    
    Gaps = namedtuple("Gaps", ["start", "mid", "end"])
    
    return Gaps(gap_start, gap_mid, gap_end)

In [ ]:
def calc_RSA_pair(pair):
    '''Calculates RSA from a pair of AF and experimental PDBs.'''
    
    # paths to PDB files
    af_path = '../data/external/AF_human/' + pair.af_name
    pdb_path = '../data/external/PDB/pdb' + pair.chain_name[:4].lower() + '.ent'
        
    chain = pair.chain_name[5]
    
    rsa_af = calc_RSA(af_path)
    rsa_pdb = calc_RSA(pdb_path)
    
    rsa_chain = [item for item in rsa_pdb if item.chain == chain]
    num_chains = len(set([item.chain for item in rsa_pdb]))
    
    # check that there are still no mid-gaps or mid-insertions when extracting sequences with freesasa
    seq_af = ''.join([item.residue for item in rsa_af])
    seq_chain = ''.join([item.residue for item in rsa_chain])

    for a in pairwise2.align.globalms(seq_af, seq_chain, 2, -1, -2, -1, 
                                      one_alignment_only = True, 
                                      penalize_end_gaps = False):
        align_af = a[0]
        align_exp = a[1]
    
    if len(get_gaps(align_af).mid) != 0 or len(get_gaps(align_exp).mid) != 0:
        print('Mid gap or insertion detected. Analysis not performed')
        
        return None
    
    # if there are no mid-gaps or mid-insertions continue
    else:
        df_af = pd.DataFrame(rsa_af, columns=('Chain_af', 'Number_af', 'Wild', 'total_abs_af', 'total_rel_af', 'main_abs_af', 'main_rel_af'))
        df_chain = pd.DataFrame(rsa_chain, columns=('Chain_exp', 'Number_exp', 'Wild', 'total_abs_exp', 'total_rel_exp', 'main_abs_exp', 'main_rel_exp'))
        
        # create new common numbering
        padding_start = max(pair.compare['gap_start'], pair.compare['insertion_start'])
        
        new_id_af = list(range(1 + pair.compare['insertion_start'], 
                               1 + pair.compare['num_identical'] + padding_start + pair.compare['gap_end']))
        new_id_chain = list(range(1 + pair.compare['gap_start'], 
                                  1 + pair.compare['num_identical']  + padding_start + pair.compare['insertion_end']))
        
        df_af['Common_number'] = new_id_af
        df_chain['Common_number'] = new_id_chain
        
        # merge results from alphafold and experimental pdb in one dataframe by joining on residue-type and common number
        df = df_af.merge(df_chain, how = 'outer', on = ['Common_number', 'Wild'])
        df['AF'] = pair.af_name
        df['Experimental'] = pair.chain_name
        df['Num_chains_exp'] = num_chains
        df['Resolution'] = pair.res
        df['Date'] = pair.date
        
        return df[['AF',
                   'Experimental',
                   'Resolution',
                   'Date',
                   'Num_chains_exp',
                   'Common_number', 
                   'Wild', 
                   'Chain_af', 
                   'Number_af', 
                   'total_abs_af',
                   'total_rel_af',
                   'main_abs_af', 
                   'main_rel_af',
                   'Chain_exp', 
                   'Number_exp',
                   'total_abs_exp',
                   'total_rel_exp',
                   'main_abs_exp',
                   'main_rel_exp']]

In [ ]:
def test_b_factor(af):
    '''Test that there is one pLDDT value for each residue'''
    
    filename = af.filename
    structure = PDBParser(QUIET = True).get_structure('AF', '../data/external/AF_human/' + filename)
    
    l = []
    for atom in structure.get_atoms():
        l.append((atom.get_bfactor(), atom.get_parent().id[1]))

    l = list(set(l))

    return len(af.seq[0]) == len(l)

In [ ]:
def get_pLDDT(af_filepath):
    '''Extracts pLDDT.'''
    
    structure = PDBParser(QUIET = True).get_structure('AF', af_filepath)
    
    l = []
    for atom in structure.get_atoms():
        l.append((atom.get_bfactor(), atom.get_parent().id[1]))

    l = list(set(l))

    return [(str(item[1]), item[0]) for item in sorted(l, key = lambda x: x[1])]

In [ ]:
def check_atoms(pdb_code):
    '''Checks that each residue is complete.'''
    
    pdb_filepath = '../data/external/PDB/pdb' + pdb_code + '.ent'
    structure = PDBParser(QUIET = True).get_structure('pdb', pdb_filepath)
    
    l = []
    for atom in structure.get_atoms():
        if atom.get_parent().id[0] == ' ':
            l.append(atom.get_name())
            
    if len(set(l)) == 1:
        print('Only one atom-type:', l[0])

    return len(set(l)) > 1    

### Classes

In [ ]:
class UniprotPair:
    def __init__(self, af_uniprot, chain_uniprot, af_name, chain_name, af_seq, chain_seq):
        self.af_uniprot = af_uniprot
        self.chain_uniprot = chain_uniprot
        self.af_name = af_name
        self.chain_name = chain_name
        self.af_seq = af_seq
        self.chain_seq = chain_seq

In [ ]:
class Alphafold:
    def __init__(self, filename, uniprot, seq, fragment, version):
        self.filename = filename
        self.uniprot = uniprot
        self.seq = seq
        self.fragment = fragment
        self.version = version

### Calculate RSA
Load list of UniProt pair instances

In [ ]:
file = open('../data/uniprot_pairs.pkl', 'rb')
uniprot_pairs = pickle.load(file)
file.close()

Read lists of pairs grouped according to overlap and resolution

In [ ]:
hi_100 = pd.read_csv('../data/group_hi_100.csv')
hi_100 = hi_100.to_dict(orient = 'records')

hi_99 = pd.read_csv('../data/group_hi_99.csv')
hi_99 = hi_99.to_dict(orient = 'records')

lo_100 = pd.read_csv('../data/group_low_100.csv')
lo_100 = lo_100.to_dict(orient = 'records')

lo_99 = pd.read_csv('../data/group_low_99.csv')
lo_99 = lo_99.to_dict(orient = 'records')

Separate UniProt pairs according to above lists

In [ ]:
hi_100_pairs = []
for pair in uniprot_pairs:
    for record in hi_100:
        if pair.af_name == record['AF'] and pair.chain_name == record['Chain']:
            hi_100_pairs.append(pair)

In [ ]:
hi_99_pairs = []
for pair in uniprot_pairs:
    for record in hi_99:
        if pair.af_name == record['AF'] and pair.chain_name == record['Chain']:
            hi_99_pairs.append(pair)

In [ ]:
lo_100_pairs = []
for pair in uniprot_pairs:
    for record in lo_100:
        if pair.af_name == record['AF'] and pair.chain_name == record['Chain']:
            lo_100_pairs.append(pair)

In [ ]:
lo_99_pairs = []
for pair in uniprot_pairs:
    for record in lo_99:
        if pair.af_name == record['AF'] and pair.chain_name == record['Chain']:
            lo_99_pairs.append(pair)

I'll include confidence values

In [6]:
file = open('../data/af_human_instances_list.pkl', 'rb') 
alphafold = pickle.load(file)
file.close()

According to Varardi, the confidence value, pLDDT, is stored in the B-factor field in the PDBs. In the following function, I extract the b-factor (one for each atom) together with the residue-type and test that there is one unique pLDDT value for each residue. 

In [ ]:
# takes a while
all([test_b_factor(af) for af in alphafold])

Yes, there is. So I'll calculate RSA and extract pLDDT values:

In [ ]:
rsa = []
for pair in hi_100_pairs:
    df = calc_RSA_pair(pair)
    if df is not None and check_atoms(pair.chain_name[:4]):
        plddt = pd.DataFrame(get_pLDDT('../data/external/AF_human/' + pair.af_name), columns = ['Number_af', 'pLDDT'])
        rsa.append(df.merge(plddt, how = 'left', on = 'Number_af'))
pd.concat(rsa).to_csv('../data/rsa_hi_100.csv', index = False)

In [ ]:
rsa = []
for pair in hi_99_pairs:
    df = calc_RSA_pair(pair)
    if df is not None and check_atoms(pair.chain_name[:4]):
        plddt = pd.DataFrame(get_pLDDT('../data/external/AF_human/' + pair.af_name), columns = ['Number_af', 'pLDDT'])
        rsa.append(df.merge(plddt, how = 'left', on = 'Number_af'))
pd.concat(rsa).to_csv('../data/rsa_hi_99.csv', index = False)

In [ ]:
rsa = []
for pair in lo_100_pairs:
    df = calc_RSA_pair(pair)
    if df is not None and check_atoms(pair.chain_name[:4]):
        plddt = pd.DataFrame(get_pLDDT('../data/external/AF_human/' + pair.af_name), columns = ['Number_af', 'pLDDT'])
        rsa.append(df.merge(plddt, how = 'left', on = 'Number_af'))
pd.concat(rsa).to_csv('../data/rsa_lo_100.csv', index = False)

In [ ]:
rsa = []
for pair in lo_99_pairs:
    df = calc_RSA_pair(pair)
    if df is not None and check_atoms(pair.chain_name[:4]):
        plddt = pd.DataFrame(get_pLDDT('../data/external/AF_human/' + pair.af_name), columns = ['Number_af', 'pLDDT'])
        rsa.append(df.merge(plddt, how = 'left', on = 'Number_af'))
pd.concat(rsa).to_csv('../data/rsa_lo_99.csv', index = False)